<a href="https://colab.research.google.com/github/Shaheerr/Activity_Recognition_WiFi/blob/main/ActivityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Note place the dataset in the root of Google drive and create these 4 folders in google drive:  
1.ActivityRecognition/  
2.ActivityRecognition/input_files  
3.ActivityRecognition/input_files_2  
4.ActivityRecognition/keras_models

In [ ]:
!tar xvzf "./drive/My Drive/Dataset.tar.gz" -C "./drive/My Drive/WifiActivityRecognition/"

In [ ]:
#Copy from "input_files" folder in root to input_files folder in drive/My drive/
cp -a input_files/ drive/My\ Drive/input_files

In [ ]:
#List the sorted data
import glob
import pandas as pd
import numpy as np
sorted(glob.glob("./drive/My Drive/Dataset/Dataset/annotation_*bed*.csv"))
sorted(glob.glob("./drive/My Drive/Dataset/Dataset/input_*bed*.csv"))
  

In [ ]:
#Rename shitty data with numbers in the start
import glob
import os
annot_data = sorted(glob.glob("./drive/My Drive/Dataset/Dataset/input_161219*.csv"))
for i in range(len(annot_data)):
  print(annot_data[i][1:39] + annot_data[i][46:])
  os.rename(annot_data[i], annot_data[i][0:39] + annot_data[i][46:])

In [ ]:
import numpy as np,numpy
import csv
import glob
import os

window_size = 1000
threshold = 60
slide_size = 200 #less than window_size!!!

def dataimport(path1, path2):

	xx = np.empty([0,window_size,90],float)
	yy = np.empty([0,8],float)

	###Input data###
	#data import from csv
	input_csv_files = sorted(glob.glob(path1))
	for f in input_csv_files:
		print("input_file_name=",f)
		data = [[ float(elm) for elm in v] for v in csv.reader(open(f, "r"))]
		tmp1 = np.array(data)
		x2 =np.empty([0,window_size,90],float)

		#data import by slide window
		k = 0
		while k <= (len(tmp1) + 1 - 2 * window_size):
			x = np.dstack(np.array(tmp1[k:k+window_size, 1:91]).T)
			x2 = np.concatenate((x2, x),axis=0)
			k += slide_size

		xx = np.concatenate((xx,x2),axis=0)
	xx = xx.reshape(len(xx),-1)

	###Annotation data###
	#data import from csv
	annotation_csv_files = sorted(glob.glob(path2))
	for ff in annotation_csv_files:
		print("annotation_file_name=",ff)
		ano_data = [[ str(elm) for elm in v] for v in csv.reader(open(ff,"r"))]
		tmp2 = np.array(ano_data)

		#data import by slide window
		y = np.zeros(((len(tmp2) + 1 - 2 * window_size)//slide_size+1,8))
		k = 0
		while k <= (len(tmp2) + 1 - 2 * window_size):
			y_pre = np.stack(np.array(tmp2[k:k+window_size]))
			bed = 0
			fall = 0
			walk = 0
			pickup = 0
			run = 0
			sitdown = 0
			standup = 0
			noactivity = 0
			for j in range(window_size):
				if y_pre[j] == "bed":
					bed += 1
				elif y_pre[j] == "fall":
					fall += 1
				elif y_pre[j] == "walk":
					walk += 1
				elif y_pre[j] == "pickup":
					pickup += 1
				elif y_pre[j] == "run":
					run += 1
				elif y_pre[j] == "sitdown":
					sitdown += 1
				elif y_pre[j] == "standup":
					standup += 1
				else:
					noactivity += 1

			if bed > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,1,0,0,0,0,0,0])
			elif fall > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,1,0,0,0,0,0])
			elif walk > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,1,0,0,0,0])
			elif pickup > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,1,0,0,0])
			elif run > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,0,1,0,0])
			elif sitdown > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,0,0,1,0])
			elif standup > window_size * threshold / 100:
				y[int(k/slide_size),:] = np.array([0,0,0,0,0,0,0,1])
			else:
				y[int(k/slide_size),:] = np.array([1,0,0,0,0,0,0,0])
			k += slide_size

		yy = np.concatenate((yy, y),axis=0)
	print(xx.shape,yy.shape)
	return (xx, yy)


#### Main ####
if not os.path.exists("input_files/"):
        os.makedirs("input_files/")

for i, label in enumerate (["walk"]):
	filepath1 = "./drive/My Drive/Dataset/Dataset/input_*" + str(label) + "*.csv"
	filepath2 = "./drive/My Drive/Dataset/Dataset/annotation_*" + str(label) + "*.csv"
	outputfilename1 = "./drive/My Drive/input_files/xx_" + str(window_size) + "_" + str(threshold) + "_" + label + ".csv"
	outputfilename2 = "./drive/My Drive/input_files/yy_" + str(window_size) + "_" + str(threshold) + "_" + label + ".csv"

	x, y = dataimport(filepath1, filepath2)
	with open(outputfilename1, "w") as f:
		writer = csv.writer(f, lineterminator="\n")
		writer.writerows(x)
	with open(outputfilename2, "w") as f:
		writer = csv.writer(f, lineterminator="\n")
		writer.writerows(y)
	print(label + "finish!")

In [ ]:
#Number of windows/samples of a particular activity in a csv file
import numpy as np
import pandas as pd
import csv

file = pd.read_csv("./drive/My Drive/input_files/yy_1000_60_bed.csv")
file = np.array(file)
np.shape(np.where(file[:,1] == 1))
#np.where(file[:,1] == 1)

In [ ]:
#Keeping Only activity Data and Discarding NoActivity

import pandas as pd
import numpy as np
import csv
import glob


def csv_write():
    window_size = 500
    threshold = 60
    print("csv file importing...")

    for i in ["standup"]:
#        xx = np.array([[ float(elm) for elm in v] for v in csv.reader(open("./input_files/xx_1000_60_" + str(i) + ".csv","r"))])
#        yy = np.array([[ float(elm) for elm in v] for v in csv.reader(open("./input_files/yy_1000_60_" + str(i) + ".csv","r"))])

#        xx = xx[::2,:]
#        yy = yy[::2,:]
        label = i
        SKIPROW = 2 #Skip every 2 rows -> overlap 800ms to 600ms  (To avoid memory error)
        num_lines = sum(1 for l in open("./drive/My Drive/input_files/xx_1000_60_" + str(i) + ".csv"))
        skip_idx = [x for x in range(1, num_lines) if x % SKIPROW !=0]

        xx = np.array(pd.read_csv("./drive/My Drive/input_files/xx_1000_60_" + str(i) + ".csv", header=None, skiprows = skip_idx))
        yy = np.array(pd.read_csv("./drive/My Drive/input_files/yy_1000_60_" + str(i) + ".csv", header=None, skiprows = skip_idx))
        print("Read Done")
        # eliminate the NoActivity Data
        #rows, cols = np.where(yy[:,0] == 1)
        rows = np.where(yy[:,0] == 1)
        #xx = np.delete(xx, rows[ np.where(cols==0)],0)
        xx = np.delete(xx, rows,0)
        print("Eliminate Done Done")
        #yy = np.delete(yy, rows[ np.where(cols==0)],0)

        xx = xx.reshape(len(xx),1000,90)
        
        # 1000 Hz to 500 Hz (To avoid memory error)
        xx = xx[:,::2,:90]

        #Rehsape Back to save in CSV
        xx = xx.reshape(-1,500*90)
        print("Downsampling Done")
        print(str(i), "finished...", "xx=", xx.shape, "yy=",  yy.shape)
        
        #Write Data in CSV Files
        outputfilename1 = "./drive/My Drive/input_files_2/xxx_" + str(window_size) + "_" + str(threshold) + "_" + label + ".csv"
        #outputfilename2 = "./drive/My Drive/input_files_2/yyy_" + str(window_size) + "_" + str(threshold) + "_" + label + ".csv"
        with open(outputfilename1, "w") as f:
          writer = csv.writer(f, lineterminator="\n")
          writer.writerows(xx)
        #with open(outputfilename2, "w") as f:
        #  writer = csv.writer(f, lineterminator="\n")
        #  writer.writerows(y)
        print(label + "finish!")
       
      
csv_write()

In [ ]:
#Keeping Only NoActivity Data and Discarding Activity

import pandas as pd
import numpy as np
import csv
import glob


def csv_write_no():
    window_size = 500
    threshold = 60
    print("csv file importing...")

    for i in ["standup"]:
#        xx = np.array([[ float(elm) for elm in v] for v in csv.reader(open("./input_files/xx_1000_60_" + str(i) + ".csv","r"))])
#        yy = np.array([[ float(elm) for elm in v] for v in csv.reader(open("./input_files/yy_1000_60_" + str(i) + ".csv","r"))])

#        xx = xx[::2,:]
#        yy = yy[::2,:]
        label = i
        SKIPROW = 2 #Skip every 2 rows -> overlap 800ms to 600ms  (To avoid memory error)
        num_lines = sum(1 for l in open("./drive/My Drive/input_files/xx_1000_60_" + str(i) + ".csv"))
        skip_idx = [x for x in range(1, num_lines) if x % SKIPROW !=0]

        xx = np.array(pd.read_csv("./drive/My Drive/input_files/xx_1000_60_" + str(i) + ".csv", header=None, skiprows = skip_idx))
        yy = np.array(pd.read_csv("./drive/My Drive/input_files/yy_1000_60_" + str(i) + ".csv", header=None, skiprows = skip_idx))
        print("Read Done")
        # eliminate the NoActivity Data
        #rows, cols = np.where(yy[:,0] == 1)
        rows = np.where(yy[:,7] == 1)
        #xx = np.delete(xx, rows[ np.where(cols==0)],0)
        xx = np.delete(xx, rows,0)
        print("Eliminate Done")
        #yy = np.delete(yy, rows[ np.where(cols==0)],0)

        xx = xx.reshape(len(xx),1000,90)
        
        # 1000 Hz to 500 Hz (To avoid memory error)
        xx = xx[:,::2,:90]

        #Rehsape Back to save in CSV
        xx = xx.reshape(-1,500*90)
        print("Downsampling Done")
        print(str(i), "finished...", "xx=", xx.shape, "yy=",  yy.shape)
        
        #Write Data in CSV Files
        #outputfilename1 = "./drive/My Drive/input_files_2/xxx_" + str(window_size) + "_" + str(threshold) + "_" + "none" + ".csv"
        outputfilename1 = "./drive/My Drive/Test_Data/xxx_" + str(window_size) + "_" + str(threshold) + "_"+ str(i) + "_" + "none" + ".csv"
        #outputfilename2 = "./drive/My Drive/input_files_2/yyy_" + str(window_size) + "_" + str(threshold) + "_" + label + ".csv"
        with open(outputfilename1, "w") as f:
          writer = csv.writer(f, lineterminator="\n")
          writer.writerows(xx)
        #with open(outputfilename2, "w") as f:
        #  writer = csv.writer(f, lineterminator="\n")
        #  writer.writerows(y)
        print("none" + "finish!")
        
csv_write_no()


In [ ]:
import pandas as pd
import numpy as np
import csv
import glob

#Create a NoActivity File from all Other NoActivity Data combined
x = np.empty(shape = [0,500,90], dtype = float)

for i in ["bed", "fall", "pickup", "run", "sitdown", "standup", "walk"]:
  print(x.shape)
  SKIPROW = 10
  num_lines = sum(1 for l in open("./drive/My Drive/input_files_2/NoActivity/xxx_500_60_" + str(i) + "_none" +".csv"))
  skip_idx = [x for x in range(1, num_lines) if x % SKIPROW !=0]
  xx = np.array(pd.read_csv("./drive/My Drive/input_files_2/NoActivity/xxx_500_60_" + str(i) + "_none" +".csv", header=None, skiprows = skip_idx))
  print('read ' + str(i) + ' done')
  xx = np.reshape(xx,[-1,500,90])
  x = np.concatenate((x,xx),axis=0)
  print('Cat ' + str(i) + ' done')
  
x = np.reshape(x,[-1,500*90])

  
outputfilename1 = "./drive/My Drive/WifiActivityRecognition/xxx_" + '500' + "_" + '60' + "_" + "none" + ".csv"
with open(outputfilename1, "w") as f:
  writer = csv.writer(f, lineterminator="\n")
  writer.writerows(x)
  

  


In [ ]:
print(x.shape)

x = np.reshape(x,[-1,500*90])

  
outputfilename1 = "./drive/My Drive/input_files_2/xxx_" + '500' + "_" + '60' + "_" + "none" + ".csv"
with open(outputfilename1, "w") as f:
  writer = csv.writer(f, lineterminator="\n")
  writer.writerows(x)

In [ ]:
#Machine Learning Model
import tensorflow as tf
import pandas as pd
import numpy as np
import glob

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM,Dropout
from tensorflow.python.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(200, input_shape=(500,90)))
model.add(BatchNormalization())
model.add(Dense(8,activation = 'softmax'))
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


model.summary()
model.fit(x, y,
          validation_split=0.05, epochs=2000, batch_size=200,shuffle=True)

model.save('./drive/My Drive/keras_model/my_model_1_e2000.h5') 

In [ ]:
#Importing Data for LSTM

import pandas as pd
import numpy as np
import glob
import csv
x = np.empty(shape = [0,500,90], dtype = float)
y = np.empty(shape = [0,8], dtype = float)
for i in ["none","bed", "fall", "pickup", "run", "sitdown", "standup", "walk"]:
  f = "./drive/My Drive/input_files_2/xxx_500_60_" + str(i) + ".csv"
  print("input_file_name=",f)
  data = [[ float(elm) for elm in v] for v in csv.reader(open(f, "r"))]
  tmp1 = np.array(data)
  tmp1 = np.reshape(tmp1,[-1,500,90])
  r,c,w = np.shape(tmp1)
  x = np.concatenate((x,tmp1),axis = 0)
  yy = np.empty([r,8],float)
  if i == "bed" :
     yy[:,:] = np.array([1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
  elif i == "fall":
     yy[:,:] = np.array([0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0])
  elif i == "pickup":
     yy[:,:] = np.array([0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0])
  elif i == "run":
     yy[:,:] = np.array([0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0])
  elif i == "sitdown":
     yy[:,:] = np.array([0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0])
  elif i == "standup":
     yy[:,:] = np.array([0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0])
  elif i == "walk":
     yy[:,:] = np.array([0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0])
  elif i == "none":
     yy[:,:] = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0])
      
  y = np.concatenate((y,yy),axis = 0)
  
  print(np.shape(x))
  print(np.shape(y))
  print(str(i) + "done")

In [ ]:
print(x.shape)


In [ ]:
#Machine Learning Model for TPU
%matplotlib inline

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os


from matplotlib import pyplot as plt
from IPython.display import clear_output

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM,Dropout
from tensorflow.python.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import BatchNormalization
from keras import metrics

print(keras.__version__)
model = Sequential()
model.add(LSTM(200, input_shape=(500,90),unit_forget_bias=True,bias_initializer="zeros",return_sequences = False))
model.add(Dense(8,activation = 'softmax'))
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001)
#optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.summary()
mc = keras.callbacks.ModelCheckpoint('./drive/My Drive/keras_model/weights{epoch:08d}.h5', 
                                     save_weights_only=True, period=50)

history = tpu_model.fit(x, y,
                        epochs=2000,
                        batch_size=128 * 8,
                        validation_split=0.2,shuffle = True,callbacks=[plot,mc])
tpu_model.save('./drive/My Drive/keras_model/tpu_model_full_keras212.h5', overwrite=True)
#tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)




In [ ]:
#Continue to Train model if interrupted

#Machine Learning Model for TPU
%matplotlib inline

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os


from matplotlib import pyplot as plt
from IPython.display import clear_output

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM,Dropout
from tensorflow.python.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(200, input_shape=(500,90),unit_forget_bias=True,bias_initializer="zeros"))
model.add(Dense(8,activation = 'softmax'))
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001)
#optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


tpu_model.load_weights('./drive/My Drive/keras_model/tpu_model_full_keras212.h5')
tpu_model.summary()

history = tpu_model.fit(x, y,
                        epochs=2000,
                        batch_size=128 * 8,
                        validation_split=0.2,shuffle = True,callbacks=[plot])
tpu_model.save('./drive/My Drive/keras_model/Final_KerasModel_2000epoch.h5', overwrite=True)

In [ ]:
tpu_model.save('./drive/My Drive/keras_model/Final_KerasModel_2000epoch.h5', overwrite=True)

In [ ]:
#Plot Loss
%matplotlib inline

import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
#Plot Learning
%matplotlib inline

import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="accuracy")
        ax2.plot(self.x, self.val_acc, label="validation accuracy")
        ax2.legend()
        
        plt.show();
        
plot = PlotLearning()

In [ ]:
#Machine Learning PRedictor for TPU
%matplotlib inline

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os


from matplotlib import pyplot as plt
from IPython.display import clear_output

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM,Dropout
from tensorflow.python.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(200, input_shape=(500,90),unit_forget_bias=True,bias_initializer="zeros"))
model.add(Dense(8,activation = 'softmax'))
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001)
#optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


tpu_model.load_weights('./drive/My Drive/keras_model/Final_KerasModel_2000epoch.h5')
tpu_model.summary()

#model_json = model.to_json()
#with open("./drive/My Drive/keras_model/model_arch.json", "w") as json_file:
#    json_file.write(model_json)
activity = tpu_model.predict(x_pred)

idx = np.argmax(activity,axis = 1);
act = ["bed", "fall", "pickup", "run", "sitdown", "standup", "walk","none"];
for i in idx :
  print(act[i])
#print(np.argmax(activity,axis = 1))
#tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)




In [ ]:
####### Load Prediction/Test Data ############

#Importing Data for LSTM

import pandas as pd
import numpy as np
import glob
import csv
x_test = np.empty(shape = [0,500,90], dtype = float)
for i in ["none"]:
  #f = "./drive/My Drive/Test_Data/xxx_500_60_" + str(i) + "1.csv"
  f = "./drive/My Drive/input_files_2/xxx_500_60_" + str(i) + ".csv"
  print("input_file_name=",f)
  data = [[ float(elm) for elm in v] for v in csv.reader(open(f, "r"))]
  x_pred = np.array(data)
  x_pred = np.reshape(x_pred,[-1,500,90])
  
  #Making it a multiple of 8 for TPU
  n=np.shape(x_pred)
  n= n[0] - n[0]%8;
  x_pred = x_pred[:n,:,:];
  
  print(np.shape(x_pred))
  print(str(i) + "done")


In [ ]:
x_pred = x_pred[2376::-1,:,:]
print(x_pred.shape)

In [ ]:
 idx = np.argmax(activity,axis = 1);
act = ["bed", "fall", "pickup", "run", "sitdown", "standup", "walk","none"];
for i in idx :
  print(act[i])

In [ ]:
#Machine Learning PRedictor for TPU
%matplotlib inline

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os


from matplotlib import pyplot as plt
from IPython.display import clear_output

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM,Dropout
from tensorflow.python.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(200, input_shape=(500,90),unit_forget_bias=True,bias_initializer="zeros"))
model.add(Dense(8,activation = 'softmax'))
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001)
#optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


tpu_model.load_weights('./drive/My Drive/keras_model/weights00001200.h5')
tpu_model.summary()

# serialize weights to HDF5
#tpu_model.save("./drive/My Drive/keras_model/tpu_model_full_keras212.hdf5")


# Save the weights
tpu_model.save_weights('./drive/My Drive/keras_model/weights_1200epoch.h5')

# Save the model architecture
model_json = model.to_json()
with open("./drive/My Drive/keras_model/model_arch.json", "w") as json_file:
    json_file.write(model_json)

    
print("Saved model to disk")




In [ ]:
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the port
server_address = ('0.0.0.0', 8090)
print('starting up on %s port %s' % server_address)
sock.bind(server_address)


# Listen for incoming connections
sock.listen(1)


while True:
    # Wait for a connection
    print('waiting for a connection')
    connection, client_address = sock.accept()
    
    
    
    try:
        print('connection from', client_address)

        # Receive the data in small chunks and retransmit it
        while True:
            data = connection.recv(1024)
            print('received "%s"' % data)
            if data:
              print('DATA STILL COMING')
                #print >>sys.stderr, 'sending data back to the client'
                #connection.sendall(data)
            else:
                print >>sys.stderr, 'no more data from', client_address
                break
            
    finally:
        # Clean up the connection
        connection.close()


In [ ]:
connection.close()

In [ ]:
import keras; 
print(keras.__version__)

In [ ]:
!pip install keras==2.1.2

In [ ]:
#Testing Matrices


import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os
SKIPROW = 2 
num_lines = sum(1 for l in open("./drive/My Drive/input_files/xx_1000_60_" + "run" + ".csv"))
skip_idx = [x for x in range(1, num_lines) if x % SKIPROW !=0]

x_run = np.array(pd.read_csv("./drive/My Drive/input_files/xx_1000_60_" + "run" + ".csv", header=None, skiprows = skip_idx))
x_no = np.array(pd.read_csv("./drive/My Drive/Test_Data/xxx_500_60_" + "run" + "1.csv", header=None))
print(x_run.shape)
print(x_no.shape)




In [ ]:
x_no = np.array(pd.read_csv("./drive/My Drive/Test_Data/xxx_500_60_" + "none" + "1.csv", header=None))


In [ ]:
print(x_run.shape)
print(x_no.shape)

print(x_run[3590-35:3590,1])
print(x_no[2381-10:2381,1])



In [ ]:
x_run = x_run.reshape(len(x_run),1000,90)
x_run = x_run[:,::2,:90]
print(x_run.shape)

In [ ]:
x_run = x_run[:3584,:,:]